# Semantic Similarity

## Steps
1. sentence -> PhoBERT Embedding -> sentence embedding
2. sentence embedding cosine similarity

3. word2vec

## Fields
1. problem - solution - domain - project_name
2. customer behavior - problem
3. outstands - solution

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
  features = phobert(input_ids) # Models outputs are now tuples

print(input_ids)

In [ ]:
from underthesea import word_tokenize

max_len=100

sentence="Chào bạn, mình là Nghi sinh viên trường Đại học Bách khoa."

# Word Segmented Sentence
sentence = word_tokenize(sentence, format="text")

# Tokenize: Padding -> Pytorch Tensor
# https://huggingface.co/transformers/v3.3.1/internal/tokenization_utils.html
sentence_tokenizer = tokenizer(text=sentence,padding='max_length', max_length=max_len) # tf, np, pt
input_ids = torch.tensor([sentence_tokenizer.input_ids])
attention_mask = sentence_tokenizer.attention_mask
print(input_ids)


In [ ]:
def phobert_tokenizer(sentence, max_len=100):
    # Word Segmented Sentence
    text = word_tokenize(sentence, format="text")
    
    encoding = tokenizer.encode_plus(
                text,
                truncation=True,
                add_special_tokens=True,
                max_length=max_len,
                padding='max_length',
                return_attention_mask=True,
                return_token_type_ids=False,
                return_tensors='pt',
            )
            
    return {
        'text': text,
        'input_ids': encoding['input_ids'], #torch.Size([1, 100])
        'attention_mask': encoding['attention_mask'], #torch.Size([1, 100])
        # 'targets': torch.tensor(label, dtype=torch.long),
    }       

In [ ]:
import numpy

# Output: the hidden state vector of pre-defined hidden size corresponding to each token in the input sequence.
encoded_sequence = phobert_tokenizer(sentence)
input_ids = encoded_sequence['input_ids']
attention_mask = encoded_sequence['attention_mask']

phobert(input_ids, attention_mask, return_dict=False) # Dropout will errors if without return_dict

## Word2Vec
[Github Word2Vec Vietnamese](https://github.com/sonvx/word2vecVN?fbclid=IwAR3JA6FwBTSotl6u_JkXBuHmaeTGRTmkWSo_zCjdqp0zArK2mUJ2tc15dvU)

In [ ]:
!pip3 install gensim pot #pot is needed

In [128]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path='word2vec/baomoi.model.bin'
word_vectors = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
word_vectors["văn_bản"]

In [155]:
#https://radimrehurek.com/gensim/models/keyedvectors.html

from underthesea import word_tokenize

docs = word_tokenize("khách sạn du lịch máy bay đặt vé nghỉ dưỡng thư giãn")
print(word_vectors.doesnt_match(docs)) #đặt

# sim = word_vectors.most_similar('yêu', topn=50)
# print(sim)

w1 = "dịch_vụ"
w2 = "khách_sạn"
result = word_vectors.similarity(w1, w2)
print(f'<{w1}> is {result} similar to <{w2}>')

# most_similar_key, similarity = result[0]  # look at the first match
# print(f"{most_similar_key}: {similarity:.4f}")

sentence_obama = word_tokenize('Obama là tổng thống da màu nỗi tiếng được phỏng vấn', format="text").lower().split()
sentence_president = 'Ngài Tổng thống chào báo chí ở Chicago'
sentence_president = word_tokenize(sentence_president, format="text").lower().split()

#Compute the Word Mover’s Distance between two documents. 
# similarity = word_vectors.wmdistance(sentence_obama, sentence_president)
# print(f"{sentence_obama}\n {sentence_president} \n -> {similarity:.4f}")


similarity = word_vectors.n_similarity(['cháo', 'li'], ['nước', 'đồ_ăn'])
print(f"n_sim -> {similarity:.4f}")


# vector = word_vectors['chén']  # numpy vector of a word
# vector.shape

# vector = word_vectors.get_vector('văn', norm=True)
# vector.shape

đặt
<dịch_vụ> is 0.23475560545921326 similar to <khách_sạn>
n_sim -> 0.2734


In [190]:
from underthesea import word_tokenize
from underthesea import pos_tag, chunk, classify

def get_nouns_verbs(doc):
  pos_str = pos_tag(sentence)
  text_group = set([word.replace(' ', '_') for word,tag,num in chunk(sentence) if tag == 'Np' or tag == 'N' or tag == 'V'])
  # print(text_group)
  return text_group

def is_document_related(doc1, doc2, word2vec):
  if word_vectors.wmdistance(doc1, doc2) > 0.5:
    return True
  word_seg_list1 = get_nouns_verbs(doc1)
  word_seg_list2 = get_nouns_verbs(doc2)

  print(word_seg_list1)

  res = []
  for w1 in word_seg_list1:
    for w2 in word_seg_list2:
      try:
        sim = word2vec.similarity(w1, w2)
        if (sim >= 0.3):
          res.append((w1, w2, sim))
      except:
        continue

  return res

In [192]:
doc1='Nhờ các công cụ đa phương tiện của máy tính như văn bản, đồ họa, hình ảnh, âm thanh, giáo viên sẽ xây dựng được bài giảng sinh động thu hút sự tập trung của người học'
doc2='Ứng dụng công nghệ thông tin vào dạy học.'

is_document_related(doc1, doc2, word_vectors)


{'âm_thanh', 'được', 'bài', 'văn_bản', 'đồ', 'học', 'máy_tính', 'giảng', 'phương_tiện', 'họa', 'tập_trung', 'sự', 'người', 'đa', 'công_cụ', 'thu_hút', 'xây_dựng', 'hình_ảnh', 'giáo_viên'}
{'Ứng_dụng', 'dạy_học', 'công_nghệ_thông_tin'}
{'âm_thanh', 'được', 'bài', 'văn_bản', 'đồ', 'học', 'máy_tính', 'giảng', 'phương_tiện', 'họa', 'tập_trung', 'sự', 'người', 'đa', 'công_cụ', 'thu_hút', 'xây_dựng', 'hình_ảnh', 'giáo_viên'}


0.5020064592761162

In [193]:
from gibberish_detector import detector

# open('data/WIKI.txt')

Detector = detector.create_from_model('vie-model/gibberish.vie.model')

def is_gibberish(doc):
  return Detector.is_gibberish(doc)

In [ ]:
class SpamFiltering:
  def __init__(self):
    self.detector = detector.create_from_model('vie-model/gibberish.vie.model')

  def is_idea_spam(self, idea_object):
    solution = idea_object.solution
    